<a href="https://colab.research.google.com/github/trajinthan/pump-it-up-data-mining/blob/main/pumb_it_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Load data**

Authenticate with google drive

In [44]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Load data from google drive to colab work space according to the csv file id

In [45]:
import pandas as pd

training_labels = drive.CreateFile({'id':'12QS3xedC7EoPS4Xj2cNVuwBSLnvbJMNM'}) 
training_labels.GetContentFile('TrainLabel.csv')  
train_label = pd.read_csv('TrainLabel.csv')

training_values = drive.CreateFile({'id':'1F4TZBjMRpTPkEbW7vjpQBIhl7Kp3QlEf'}) 
training_values.GetContentFile('TrainValue.csv')  
train_value = pd.read_csv('TrainValue.csv')

testing_labels = drive.CreateFile({'id':'1Y4Idhc-WeUTM5uQSjZOqyQ5r4ePgUD84'}) 
testing_labels.GetContentFile('TestData.csv')  
Xtest = pd.read_csv('TestData.csv')

Merge training data values and respective training data labels

In [46]:
train_data = train_value.merge(train_label, on='id')

In [47]:
train_data.head().T

,0,1,2,3,4
id,69572,8776,34310,67743,19728
amount_tsh,6000,0,25,0,0
date_recorded,2011-03-14,2013-03-06,2013-02-25,2013-01-28,2011-07-13
funder,Roman,Grumeti,Lottery Club,Unicef,Action In A
gps_height,1390,1399,686,263,0
installer,Roman,GRUMETI,World vision,UNICEF,Artisan
longitude,34.9381,34.6988,37.4607,38.4862,31.1308
latitude,-9.85632,-2.14747,-3.82133,-11.1553,-1.82536
wpt_name,none,Zahanati,Kwa Mahundi,Zahanati Ya Nanyumbu,Shuleni
num_private,0,0,0,0,0


Get the data types of the values

In [48]:
train_data.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

Get the count of null values in the training data set

In [49]:
train_data.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [50]:
train_data['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

# **Drop identical or unnecessary columns**

1. The features **quantity** and **quantity_group** are described as **The quantity of water** So we need to check whether they are same in or not

In [51]:
train_data['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [52]:
train_data['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [53]:
train_data.groupby(['quantity','quantity_group']).count()

,,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
quantity,quantity_group,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
dry,dry,6246,6246,6246,5781,6246,5782,6246,6246,6246,6246,6246,6087,6246,6246,6246,6246,6246,6246,5786,6246,5974,3546,5836,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246,6246
enough,enough,33186,33186,33186,31963,33186,31964,33186,33186,33186,33186,33186,33087,33186,33186,33186,33186,33186,33186,31366,33186,31610,18363,31979,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186,33186
insufficient,insufficient,15129,15129,15129,13950,15129,13940,15129,15129,15129,15129,15129,15020,15129,15129,15129,15129,15129,15129,14569,15129,13710,8053,13934,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129,15129
seasonal,seasonal,4050,4050,4050,3414,4050,3415,4050,4050,4050,4050,4050,4046,4050,4050,4050,4050,4050,4050,3872,4050,3740,932,3901,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050,4050
unknown,unknown,789,789,789,657,789,644,789,789,789,789,789,789,789,789,789,789,789,789,473,789,489,340,694,789,789,789,789,789,789,789,789,789,789,789,789,789,789,789,789


As both features carry identical values we can drop either **quantity** or **quantity_group**.

2. The features **water_quality** and **quality_group** are described as **The quality of the water** So we need to check whether they are same in or not.

In [54]:
train_data['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [55]:
train_data['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [56]:
train_data.groupby(['water_quality','quality_group']).count()

,,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
water_quality,quality_group,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
coloured,colored,490,490,490,391,490,391,490,490,490,490,490,490,490,490,490,490,490,490,478,490,431,123,490,490,490,490,490,490,490,490,490,490,490,490,490,490,490,490,490
fluoride,fluoride,200,200,200,181,200,176,200,200,200,200,200,200,200,200,200,200,200,200,197,200,188,138,186,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
fluoride abandoned,fluoride,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,16,17,14,1,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
milky,milky,804,804,804,788,804,785,804,804,804,804,804,804,804,804,804,804,804,804,736,804,344,44,801,804,804,804,804,804,804,804,804,804,804,804,804,804,804,804,804
salty,salty,4856,4856,4856,4803,4856,4801,4856,4856,4856,4856,4856,4638,4856,4856,4856,4856,4856,4856,4485,4856,4419,1453,4706,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856,4856
salty abandoned,salty,339,339,339,331,339,331,339,339,339,339,339,338,339,339,339,339,339,339,239,339,332,62,329,339,339,339,339,339,339,339,339,339,339,339,339,339,339,339,339
soft,good,50818,50818,50818,47945,50818,47948,50818,50818,50818,50818,50818,50666,50818,50818,50818,50818,50818,50818,48437,50818,48262,28629,48416,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818,50818
unknown,unknown,1876,1876,1876,1309,1876,1296,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1478,1876,1533,784,1399,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876,1876


As both features have almost same values we can drop either one of them. **water_quality** is more informative.

3. The features **payment** and **payment_type** are described as **What the water costs** So we need to check whether they are same in or not.

In [57]:
train_data['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [58]:
train_data['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

In [59]:
train_data.groupby(['payment','payment_type']).count()

,,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
payment,payment_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
never pay,never pay,25348,25348,25348,23440,25348,23443,25348,25348,25348,25348,25348,25246,25348,25348,25348,25348,25348,25348,24594,25348,23604,11385,23776,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348,25348
other,other,1054,1054,1054,1038,1054,1039,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,977,1054,1028,220,995,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054,1054
pay annually,annually,3642,3642,3642,3533,3642,3529,3642,3642,3642,3642,3642,3591,3642,3642,3642,3642,3642,3642,3626,3642,3604,2355,3585,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642,3642
pay monthly,monthly,8300,8300,8300,8084,8300,8083,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8226,8300,7877,5970,8221,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300,8300
pay per bucket,per bucket,8985,8985,8985,8717,8985,8711,8985,8985,8985,8985,8985,8767,8985,8985,8985,8985,8985,8985,8552,8985,8816,6707,8700,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985,8985
pay when scheme fails,on failure,3914,3914,3914,3869,3914,3872,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3896,3914,3793,1264,3777,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914,3914
unknown,unknown,8157,8157,8157,7084,8157,7068,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,6195,8157,6801,3333,7290,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157,8157


As both features carry identical values we can drop either **payment** or **payment_type**.

4. The features **waterpoint_type** and **waterpoint_type_group** are described as **The kind of waterpoint** So we need to check whether they are same in or not.

In [60]:
train_data['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [61]:
train_data['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

In [62]:
train_data.groupby(['waterpoint_type','waterpoint_type_group']).count()

,,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,status_group
waterpoint_type,waterpoint_type_group,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cattle trough,cattle trough,116,116,116,113,116,110,116,116,116,116,116,116,116,116,116,116,116,116,114,116,113,71,88,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116
communal standpipe,communal standpipe,28522,28522,28522,25778,28522,25790,28522,28522,28522,28522,28522,28407,28522,28522,28522,28522,28522,28522,27196,28522,28030,23049,26368,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522,28522
communal standpipe multiple,communal standpipe,6103,6103,6103,6064,6103,6063,6103,6103,6103,6103,6103,5859,6103,6103,6103,6103,6103,6103,5815,6103,6032,5218,6051,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103,6103
dam,dam,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,5,6,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
hand pump,hand pump,17488,17488,17488,16997,17488,16977,17488,17488,17488,17488,17488,17485,17488,17488,17488,17488,17488,17488,16255,17488,15368,1468,16978,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488,17488
improved spring,improved spring,784,784,784,743,784,748,784,784,784,784,784,784,784,784,784,784,784,784,773,784,701,83,780,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784,784
other,other,6380,6380,6380,6063,6380,6050,6380,6380,6380,6380,6380,6371,6380,6380,6380,6380,6380,6380,5906,6380,5272,1340,6073,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380,6380


As both features have almost same values we can drop either one of them. **waterpoint_type** is more informative.